In [ ]:
#import dependencies
!pip install requests

import requests
import re
import pandas as pd

pd.set_option('display.max_colwidth', None)

#Collecting Data: Hashtags from **Liked List Posts**

In [ ]:
#import tiktok data into notebook
tiktok_df = pd.read_json("/content/user_data.json")

#export hashtags,date and link from each liked post into a dataframe
likedVids_df = pd.DataFrame(columns=['Link','Liked Date','Tags'])

In [ ]:
#get tiktok urls
links = []
html_contents = []
for vid in tiktok_df['Activity']['Like List']['ItemFavoriteList']:
    links.append(vid['Link'])

In [ ]:
#get html content of each tiktok url
for link in links:
  print('fetching from vid num: ' + str(links.index(link)))
  response = requests.get(link)
  html_contents.append(response.content)

fetching from vid num: 0
fetching from vid num: 1
fetching from vid num: 2
fetching from vid num: 3
fetching from vid num: 4
fetching from vid num: 5
fetching from vid num: 6
fetching from vid num: 7
fetching from vid num: 8
fetching from vid num: 9
fetching from vid num: 10
fetching from vid num: 11
fetching from vid num: 12
fetching from vid num: 13
fetching from vid num: 14
fetching from vid num: 15
fetching from vid num: 16
fetching from vid num: 17
fetching from vid num: 18
fetching from vid num: 19
fetching from vid num: 20
fetching from vid num: 21
fetching from vid num: 22
fetching from vid num: 23
fetching from vid num: 24
fetching from vid num: 25
fetching from vid num: 26
fetching from vid num: 27
fetching from vid num: 28
fetching from vid num: 29
fetching from vid num: 30
fetching from vid num: 31
fetching from vid num: 32
fetching from vid num: 33
fetching from vid num: 34
fetching from vid num: 35
fetching from vid num: 36
fetching from vid num: 37
fetching from vid num:

/usr/local/lib/python3.10/dist-packages/urllib3/connection.py:223: RuntimeWarning: coroutine 'make_requests' was never awaited
  if not any(isinstance(v, str) and v == SKIP_HEADER for v in values):
/usr/local/lib/python3.10/dist-packages/urllib3/connection.py:223: RuntimeWarning: coroutine 'make_api_calls' was never awaited
  if not any(isinstance(v, str) and v == SKIP_HEADER for v in values):
/usr/local/lib/python3.10/dist-packages/urllib3/connection.py:223: RuntimeWarning: coroutine 'get_symbols' was never awaited
  if not any(isinstance(v, str) and v == SKIP_HEADER for v in values):


Streaming output truncated to the last 5000 lines.
fetching from vid num: 2736
fetching from vid num: 2737
fetching from vid num: 2738
fetching from vid num: 2739
fetching from vid num: 2740
fetching from vid num: 2741
fetching from vid num: 2742
fetching from vid num: 2743
fetching from vid num: 2744
fetching from vid num: 2745
fetching from vid num: 2746
fetching from vid num: 2747
fetching from vid num: 2748
fetching from vid num: 2749
fetching from vid num: 2750
fetching from vid num: 2751
fetching from vid num: 2752
fetching from vid num: 2753
fetching from vid num: 2754
fetching from vid num: 2755
fetching from vid num: 2756
fetching from vid num: 2757
fetching from vid num: 2758
fetching from vid num: 2759
fetching from vid num: 2760
fetching from vid num: 2761
fetching from vid num: 2762
fetching from vid num: 2763
fetching from vid num: 2764
fetching from vid num: 2765
fetching from vid num: 2766
fetching from vid num: 2767
fetching from vid num: 2768
fetching from vid num: 27

In [ ]:
#check to see if all html contents were received
links_series = pd.Series(links)
htmlContents_series = pd.Series(html_contents)

print(len(links_series))
print(len(htmlContents_series))



In [ ]:
#export htmlContents for later use.
htmlContents_series.to_csv('htmlContents.csv',index=False)

In [ ]:
#get the tags from the html content. Each tiktok post html follows same format in displaying tag. They exist within a .../tag/=<hashtag_name>...
pattern = r"/tag/(.*?)(?=\")"
tags = []
print(type(htmlContents_series[0]))
for html in htmlContents_series:
    substrings = re.findall(pattern, html.decode('utf-8'))
    tags.append(substrings)

<class 'bytes'>


In [ ]:
#check if my supposed method for getting tags is right
tags_series = pd.Series(tags)
print(tags[0])

['metalband', 'musicrecommendations', 'nu_metal', 'indianfolkmetal', 'metal', 'musictok', 'indianmetal', 'playlistrecommendations', 'politicalsong', 'socialissues']


In [ ]:
#get the date at which I liked every post
dates = []
for vid in tiktok_df['Activity']['Like List']['ItemFavoriteList']:
  dates.append(vid['Date'])


In [ ]:
#combine series into a single dataframe
dates_series = pd.Series(dates)
likedVids_info_df = pd.concat([links_series,tags_series,dates_series],axis=1)
print(likedVids_info_df.loc[231])

In [ ]:
#export dataframe as csv for later use and visualisation in powerbi
likedVids_info_df.to_csv('likedVids_df.csv',index=False)
test_df = pd.read_csv('/content/likedVids_df.csv')

#assure there is no missing data when converting dataframe into csv
print(test_df.loc[0])

#Collecting Data: Sentiment Analysis on **Comments**

In [ ]:
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.2 MB/s eta 0:00:00


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
comments_df = pd.DataFrame(columns=['Comment','Pos. Score','Neg. Score','Neu. Score', 'Comp. Score'])

In [ ]:

analyzer = SentimentIntensityAnalyzer()

#store the comment and corresponding sentiment scores in above dataframe
for comment in tiktok_df['Comment']['Comments']['CommentsList']:
  text = comment['Comment']
  sentiment_scores = analyzer.polarity_scores(text)

  sentiment_entry = {
      'Comment': text,
      'Pos. Score': sentiment_scores['pos'],
      'Neg. Score': sentiment_scores['neg'],
      'Neu. Score': sentiment_scores['neu'],
      'Comp. Score': sentiment_scores['compound']
  }
  comments_df = comments_df.append(sentiment_entry,ignore_index=True)


In [ ]:
#export for visualisation in powerbi
comments_df.to_csv('comments_analysis.csv',index=False)

In [ ]:
!pip install textblob

In [ ]:
import pandas as pd
tiktok_df = pd.read_json('/content/user_data.json')
comments = tiktok_df['Comment']['Comments']['CommentsList']

In [ ]:
from textblob import TextBlob

#store sentiment scores of each comment
textblob_sentiments = pd.DataFrame(columns=['Sentiment Score'])
for comment in comments:
  text = comment['Comment']
  blob = TextBlob(text)

  polarity = blob.sentiment.polarity
  subjectivity = blob.sentiment.subjectivity
  sentiment = "Positive" if polarity > 0 else "Negative" if polarity < 0 else "Neutral"
  textblob_sentiments = textblob_sentiments.append({'Sentiment Score':sentiment},ignore_index=True)

print(textblob_sentiments)


In [ ]:
#export for visualisation in powerbi
textblob_sentiments.to_csv('textblob_sentiments.csv',index=False)

Neutral     566
Positive    146
Negative    105
Name: Sentiment Score, dtype: int64


#Collecting Data: Login History

In [ ]:
#get datetime of each login and hour in another column to implement slicer in powerbi later
login_df = pd.DataFrame(columns=['Date','Hour'])
pattern = r"\d{2}:\d{2}:\d{2}"

for login in tiktok_df['Activity']['Login History']['LoginHistoryList']:
  hour = login['Date'][11:13]
  login_details = {
      'Date': login['Date'],
      'Hour': hour
  }
  login_df = login_df.append(login_details,ignore_index=True)

login_df.to_csv('login_info.csv',index=False)

#Cleaning Data: Login History


---
Grouping every row by day, then counting the number of rows in each grouping. A.K.A , login frequency by day.


In [ ]:
import pandas as pd

In [ ]:
login_df = pd.read_csv('/content/login_info.csv')

print(login_df.head())

                      Date  Hour
0  2023-05-09 20:16:21 UTC    20
1  2023-05-09 20:10:15 UTC    20
2  2023-05-09 20:04:40 UTC    20
3  2023-05-09 19:45:35 UTC    19
4  2023-05-09 18:14:53 UTC    18


In [ ]:
#all date formats are the same, get the year month day ONLY by getting first 11 characters.

def get_first_11_chars(s):
    return s[:11]

login_df['Date(y/m/d)'] = login_df['Date'].apply(get_first_11_chars)

#group rows with the same date, count the number of rows within each group.
login_frequency = login_df.groupby('Date(y/m/d)').size().reset_index(name='times logged in')

print(login_frequency)

     Date(y/m/d)  times logged in
0    2022-11-22                20
1    2022-11-23                47
2    2022-11-24                47
3    2022-11-25                59
4    2022-11-26                49
..           ...              ...
145  2023-05-05                59
146  2023-05-06                54
147  2023-05-07                73
148  2023-05-08                59
149  2023-05-09                49

[150 rows x 2 columns]


In [ ]:
login_frequency.to_csv('login_frequency.csv',index=False)

#Cleaning Data: Liked List


---
Map hashtags to their occurences in the posts

In [ ]:
likedVids_df = pd.read_csv('/content/likedVids_df.csv')

print(likedVids_df)

                                                      0  \
0     https://www.tiktokv.com/share/video/7235826205...   
1     https://www.tiktokv.com/share/video/7233904237...   
2     https://www.tiktokv.com/share/video/7234715360...   
3     https://www.tiktokv.com/share/video/7234450743...   
4     https://www.tiktokv.com/share/video/7235463779...   
...                                                 ...   
7731  https://www.tiktokv.com/share/video/6809070567...   
7732  https://www.tiktokv.com/share/video/6809375411...   
7733  https://www.tiktokv.com/share/video/6808924462...   
7734  https://www.tiktokv.com/share/video/6808924125...   
7735  https://www.tiktokv.com/share/video/6807446632...   

                                                      1                    2  
0     ['metalband', 'musicrecommendations', 'nu_meta...  2023-05-22 23:34:50  
1                  ['dog', 'dogsoftiktok', 'rescuedog']  2023-05-21 20:01:01  
2                                                    [

In [ ]:
from ast import literal_eval

# Create an empty dictionary to store hashtag counts
tag_frequency = {}

#in csv, list of hashtags got converted into string, use literal_eval to convert back into a list of strings
for tags in likedVids_df['1']:
  decoded_tags = literal_eval(tags)

  for tag in decoded_tags:
    if tag in tag_frequency:
      tag_frequency[tag] += 1
    else:
      tag_frequency[tag] = 1


In [ ]:
#convert the tag_frequency dict to a dataframe
tagFrequency_df = pd.DataFrame(tag_frequency.items(),columns =  ['tag','frequency']).sort_values('frequency',ascending=False)

In [ ]:
print(tagFrequency_df)

                 tag  frequency
17               fyp       1886
38            foryou        772
70             viral        527
45        foryoupage        477
18              fypシ        415
...              ...        ...
3830  bandofbrothers          1
3829        maneskin          1
3828            rosé          1
3826      avanigregg          1
9079       gamingszn          1

[9080 rows x 2 columns]


In [ ]:
tagFrequency_df.to_csv('tagFrequency.csv',index=False)


#Collecting Data: Dates from Comments

In [ ]:
import pandas as pd

tiktok_deets = pd.read_json('/content/user_data.json')

dates = pd.DataFrame(columns=['date'])
for comment in tiktok_deets['Comment']['Comments']['CommentsList']:
  dates = dates.append({'date': comment['Date']},ignore_index=True)

In [ ]:
def remove_last_n_chars(string, n):
    return string[:-n]

dates['date'] = dates['date'].apply(lambda x: remove_last_n_chars(x, 9))


In [ ]:
print(dates.head())

         date
0  2023-04-29
1  2023-04-29
2  2023-04-25
3  2023-04-14
4  2023-04-12


In [ ]:
dates.to_csv('comment_dates.csv',index=False)